In [1]:
# import pycsdp.utilites.pca
from pycsdp.utilites import pca
import numpy as np
from joblib import Parallel,delayed
import multiprocessing
import autograd.numpy as anp
from pymoo.model.problem import Problem
import math
from pymoo.algorithms.hfidea_d_non_dom import HFIDEAD
from pymoo.algorithms.LHFiD import LHFID
from pymoo.algorithms.A2_LHFiD import A2_LHFID
from pymoo.optimize import minimize
from pymoo.factory import get_performance_indicator
import pandas as pd
from pymoo.util.normalization import normalize as norm
import hvwfg
from pymoo.util.function_loader import load_function
from pymoo.factory import get_problem, get_crossover, get_mutation, get_reference_directions, get_selection
seeds = 1 + np.arange(5)
print(seeds)
def get_ref(M):
    if M==2:
        return get_reference_directions('das-dennis',2,n_partitions=99)
    elif M==3:
        return get_reference_directions('das-dennis',3,n_partitions=13)
    elif M==4:
        return get_reference_directions('das-dennis',4,n_partitions=8)
    elif M==5:
        n_partitions = 6
        return get_reference_directions('das-dennis',5,n_partitions=6), 6
    elif M==6:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 6, n_partitions=4, scaling=1.0), get_reference_directions("das-dennis", 6, n_partitions=4, scaling=0.5))
    elif M==7:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 7, n_partitions=4, scaling=1.0), get_reference_directions("das-dennis", 7, n_partitions=3, scaling=0.5))
    elif M==8:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 8, n_partitions=3, scaling=1.0), get_reference_directions("das-dennis", 8, n_partitions=3, scaling=0.5))
    elif M==9:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 9, n_partitions=3, scaling=1.0), get_reference_directions("das-dennis", 9, n_partitions=3, scaling=0.5))
    elif M==10:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 10, n_partitions=3, scaling=1.0), get_reference_directions("das-dennis", 10, n_partitions=3, scaling=0.5))
    elif M==11:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 11, n_partitions=2, scaling=1.0), get_reference_directions("das-dennis", 11, n_partitions=2, scaling=0.5))
    elif M==12:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 12, n_partitions=2, scaling=1.0), get_reference_directions("das-dennis", 12, n_partitions=2, scaling=0.5))
    elif M==13:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 13, n_partitions=2, scaling=1.0), get_reference_directions("das-dennis", 13, n_partitions=1, scaling=0.5))
    elif M==14:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 14, n_partitions=2, scaling=1.0), get_reference_directions("das-dennis", 14, n_partitions=1, scaling=0.5))
    elif M==15:
        return get_reference_directions("multi-layer", get_reference_directions("das-dennis", 15, n_partitions=2, scaling=1.0), get_reference_directions("das-dennis", 15, n_partitions=1, scaling=0.5))
for i in range(2,16):
    print(i,len(get_ref(i)))
print("worked")

[1 2 3 4 5]
2 100
3 105
4 165
5 2
6 252
7 294
8 240
9 330
10 440
11 132
12 156
13 104
14 119
15 135
worked


In [2]:
from pymoo.model.callback import Callback
class MyCallback(Callback):

    def __init__(self) -> None:
        super().__init__()
        self.data["pop"] = []
#         self.data["nadir-update"] = []
#         self.data["insights"] = []
#         self.data["term-gen"] = []

    def notify(self, algorithm):
        self.data["pop"].append(algorithm.pop)
#         self.data["nadir-update"].append(algorithm.term_gen)
#         self.data["insights"].append(algorithm.data)
#         self.data["term-gen"] = algorithm.termination_suggestion

class MyCallback1(Callback):

    def __init__(self) -> None:
        super().__init__()
        self.data["pop"] = []
#         self.data["nadir-update"] = []
#         self.data["insights"] = []
        self.data["term-gen"] = []

    def notify(self, algorithm):
        self.data["pop"].append(algorithm.pop)
#         self.data["nadir-update"].append(algorithm.term_gen)
#         self.data["insights"].append(algorithm.data)
        self.data["term-gen"] = algorithm.termination_suggestion

In [3]:
import autograd.numpy as anp
import math

from pymoo.model.problem import Problem

import autograd.numpy as anp
from pymoo.model.problem import Problem
class DTLZ5_3_5(Problem):

    def __init__(self, ess):

        # define lower and upper bounds -  1d array with length equal to number of variable
        xl = 0 * anp.ones(14)
        xu = 1 * anp.ones(14)

        super().__init__(n_var=14, n_obj=len(ess), n_constr=3, xl=xl, xu=xu, elementwise_evaluation=True)
        self.ess = ess

    def _evaluate(self, x, out, *args, **kwargs):
        M = 5
        I = 3
        noc = M-I+1
        k = 1
        p = np.zeros(M-I+1)
        p = [(M-I+2)-(i+1) for i in range(len(p))]
        p[0] = M-1
        g = sum([(x[i]-0.5)**2 for i in range(M-1,M+k-1)])
        theta = np.zeros(M-1)
        for i in range(I-1):
            theta[i] = math.pi*x[i]/2
        for i in range(I-1,M-1):
            theta[i] = math.pi*(1+2*g*x[i])/(4*(1+g))
        F = np.zeros(M)
#         val = 0.5*(1+g)
#         for i in range(M-1):
#             val = val*math.cos(theta[i])
#         F[0] = val
#         F[M-1] = (1+g)*math.sin(theta[0])
#         for i in range(1,M-1):
#             F[i] = (1+g)*math.sin(theta[M-(i+1)])
#             for j in range(M-i):
#                 F[i] = F[i]*math.cos(theta[j])
        for i in range(M):
            if i==0:
                sine = 1
            else:
                sine = math.sin(theta[M-1-i])
            coss = 1
            for j in range(M-1-i):
                coss = coss*math.cos(theta[j])
            F[i] = (1+g)*coss*sine
                
                
                
        G = np.zeros(len(p))
#         for i in range(len(p)):
#             val = 0
#             for j in range(I-2+1):
#                 val += F[M-1-(j)]**2
#             G[i] = 1 - val - (2**p[i])*(F[i]**2)
            
        ind = 0
        for j in range(I-1):
            ind += F[M-j-1]**2
        for i in range(M-I+1):
            if i==0:
                p=M-I
            else:
                p=M-I+1-i
            G[i] = -ind +1 -(2**p)*(F[i]**2)

        out["F"] = anp.column_stack(F)[:,self.ess]
        out["G"] = anp.column_stack(G)

In [4]:
"""Set the number of objectives"""
M = 5


## Parameters for A2 LHFiD
del_after = 40
del_threshold = 40
decimal_del = 3

## TODO here

In [5]:
ngen= 10000

names = ['DTLZ5_3_5'] # change problem name

for i in range(1):
    # building the pca libraries (nl_pca or l_pca)
    nl_fn, free_fs = pca.pca_reduction('nl_pca','Release')
    #######
    algo = 'A_LHFID'
    name = names[i]
    n_obj = 5 # change number of objectives
    print(name, ngen)
    crossover = get_crossover('real_sbx',prob=0.9,eta=20)
    mutation = get_mutation('real_pm',prob=1/15,eta=20)
    ref_dirs, n_partitions = get_ref(M)


    ################# A2 LHFiD
    # algorithm = A2_LHFID(pop_size=len(ref_dirs),ref_dirs=ref_dirs,crossover=crossover,selection=get_selection('random'),
    #                      mutation=mutation,eliminate_duplicates=True, n_partitition=n_partitions,
    #                      num_objs=M, num_ini_rvs=len(ref_dirs), deletion_gen=del_after, del_threshold=del_threshold, decimal_del=decimal_del)

    ################# OG LHFiD
    # algorithm = LHFID(pop_size=len(ref_dirs),ref_dirs=ref_dirs,crossover=crossover,selection=get_selection('random'),
    #                                mutation=mutation,eliminate_duplicates=True)
    
    algorithm = LHFID(pop_size=len(ref_dirs),ref_dirs=ref_dirs,crossover=crossover,selection=get_selection('random'),
                    mutation=mutation,eliminate_duplicates=True)

    def optimize(seed_value):
        global problem,algorithm,ngen,n_obj,name
        ess_init = [i for i in range(n_obj)]
        pfs = []
        term_gens = []
        for interation in range(n_obj):
            problem = DTLZ5_3_5(ess_init) #change problem name
            M = len(ess_init)
            ref_dirs, n_partitions = get_ref(M)

            ################# A2 LHFiD
            # algorithm = A2_LHFID(pop_size=len(ref_dirs),ref_dirs=ref_dirs,crossover=crossover,selection=get_selection('random'),
            #                      mutation=mutation,eliminate_duplicates=True, n_partitition=n_partitions,
            #                      num_objs=M, num_ini_rvs=len(ref_dirs), deletion_gen=del_after, del_threshold=del_threshold, decimal_del=decimal_del)

            ################# OG LHFiD
            algorithm = LHFID(pop_size=len(ref_dirs),ref_dirs=ref_dirs,crossover=crossover,selection=get_selection('random'),
                              mutation=mutation,eliminate_duplicates=True)
            res = minimize(problem,algorithm,('n_gen', ngen),callback=MyCallback1(),seed=seed_value)
            df = np.array(res.algorithm.callback.data["pop"])
            pf = df[-1]
            pfs.append(pf)
            term_gens.append(len(df))
            pf = np.array([ind.F for ind in pf])

            # filter only the natural cluster representatives - check the number of clusters and take the initial solutions
            pfn = norm(pf)
            dist_matrix = load_function('calc_perpendicular_distance')(pfn, ref_dirs) # i,j where i is solution and j is RV
            assoc_index = np.array([np.argmin(row) for row in dist_matrix])
            count = len(np.unique(assoc_index))
            print(count)
            pf = pf[:count]

            # np.savetxt('pf_data_nl.txt', pf)
            # out = !/home/sukrit/Documents/HFiDEA/NL_MVU_PCA/Release/nl_mvu_pca_dmoss_kemoss ./pf_data_nl.txt
            # out = out[0][7:]
            # out = out.split(' ')
            # calling the nl_pca function
            out = pca.nl_pca_reduction(pf, nl_fn, free_fs)
            print(out)
            ess = np.array([int(row)-1 for row in out])
            if len(ess_init)==len(ess):
                break
            else:
                ess_init = np.array(ess_init)
                ess_init = ess_init[ess]
        np.save('data/'+name+'-fronts-'+str(seed_value)+'.npy',pfs)
        np.save('data/'+name+'-gens-'+str(seed_value)+'.npy',term_gens)
    for seed in seeds:
        optimize(seed)
    # clean C libraries
    pca._clean_pca('nl_pca')

make: Entering directory '/home/suntzunami/anaconda3/envs/lbp2/lib/python3.9/site-packages/pycsdp/pca/nl_mvu_pca_dmoss_kemoss/Release'
rm -rf  ./csdp_library/Fnorm.o ./csdp_library/add_mat.o ./csdp_library/addscaledmat.o ./csdp_library/allocmat.o ./csdp_library/calc_dobj.o ./csdp_library/calc_pobj.o ./csdp_library/chol.o ./csdp_library/copy_mat.o ./csdp_library/easysdp.o ./csdp_library/freeprob.o ./csdp_library/initparams.o ./csdp_library/initsoln.o ./csdp_library/linesearch.o ./csdp_library/make_i.o ./csdp_library/makefill.o ./csdp_library/mat_mult.o ./csdp_library/mat_multsp.o ./csdp_library/matvec.o ./csdp_library/norms.o ./csdp_library/op_a.o ./csdp_library/op_at.o ./csdp_library/op_o.o ./csdp_library/packed.o ./csdp_library/psd_feas.o ./csdp_library/qreig.o ./csdp_library/readprob.o ./csdp_library/readsol.o ./csdp_library/sdp.o ./csdp_library/solvesys.o ./csdp_library/sortentries.o ./csdp_library/sym_mat.o ./csdp_library/trace_prod.o ./csdp_library/tweakgap.o ./csdp_library/user_e

/home/suntzunami/Desktop/obj_red/pymoo/model/problem.py:418: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif _type is "starmap":
/home/suntzunami/Desktop/obj_red/pymoo/model/problem.py:418: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif _type is "starmap":
/home/suntzunami/Desktop/obj_red/pymoo/model/problem.py:418: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif _type is "starmap":
/home/suntzunami/Desktop/obj_red/pymoo/model/problem.py:418: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif _type is "starmap":
/home/suntzunami/Desktop/obj_red/pymoo/model/problem.py:418: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif _type is "starmap":
/home/suntzunami/Desktop/obj_red/pymoo/model/problem.py:418: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif _type is "starmap":
/home/suntzunami/Desktop/obj_red/pymoo/model/problem.py:418: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif _type is "starmap":

KeyboardInterrupt: 

In [11]:

for i in range(5):
    nl_fn, free_fs = pca.pca_reduction('nl_pca','Release')

    pf = np.load("old_data/DTLZ5_3_5-tosend-1.npy")


    out = pca.nl_pca_reduction(pf, nl_fn, free_fs)
    print(out)


    pca._clean_pca('nl_pca')

make: Entering directory '/home/suntzunami/anaconda3/envs/lbp2/lib/python3.9/site-packages/pycsdp/pca/nl_mvu_pca_dmoss_kemoss/Release'
rm -rf  ./csdp_library/Fnorm.o ./csdp_library/add_mat.o ./csdp_library/addscaledmat.o ./csdp_library/allocmat.o ./csdp_library/calc_dobj.o ./csdp_library/calc_pobj.o ./csdp_library/chol.o ./csdp_library/copy_mat.o ./csdp_library/easysdp.o ./csdp_library/freeprob.o ./csdp_library/initparams.o ./csdp_library/initsoln.o ./csdp_library/linesearch.o ./csdp_library/make_i.o ./csdp_library/makefill.o ./csdp_library/mat_mult.o ./csdp_library/mat_multsp.o ./csdp_library/matvec.o ./csdp_library/norms.o ./csdp_library/op_a.o ./csdp_library/op_at.o ./csdp_library/op_o.o ./csdp_library/packed.o ./csdp_library/psd_feas.o ./csdp_library/qreig.o ./csdp_library/readprob.o ./csdp_library/readsol.o ./csdp_library/sdp.o ./csdp_library/solvesys.o ./csdp_library/sortentries.o ./csdp_library/sym_mat.o ./csdp_library/trace_prod.o ./csdp_library/tweakgap.o ./csdp_library/user_e

: 

: 